<a href="https://colab.research.google.com/github/mars241/CheckPoints/blob/main/Corrig%C3%A9_2023_04_11_Livecoding_API_%26_Geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd

# **API : disponibilités des Vélib en temps réel**

### **1. Requêtage**

In [ ]:
# Télécharger un CSV statique : le dataframe est exploitable mais pas de mise à jour automatique
link_csv = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

df_csv = pd.read_csv(link_csv, sep=";")
df_csv

In [ ]:
df_csv.columns

Index(['Identifiant station', 'Nom station', 'Station en fonctionnement',
       'Capacité de la station', 'Nombre bornettes libres',
       'Nombre total vélos disponibles', 'Vélos mécaniques disponibles',
       'Vélos électriques disponibles', 'Borne de paiement disponible',
       'Retour vélib possible', 'Actualisation de la donnée',
       'Coordonnées géographiques', 'Nom communes équipées',
       'Code INSEE communes équipées'],
      dtype='object')

In [ ]:
# parenthèse avec un exemple de csv imbriqué : les Vélo'v (colonnes total_stands et main_stands)
# ici le passage par le fichier JSON serait indispensable
link_csv_velov = "https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.csv?maxfeatures=-1"
df_csv_velov = pd.read_csv(link_csv_velov, sep=";")
df_csv_velov

In [ ]:
# Retour à Paris :
# API DYNAMIQUE : fichier json complet + mise à jour automatique
link_json = "https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilite-en-temps-reel&q=&rows=-1&facet=name&facet=is_installed&facet=is_renting&facet=is_returning&facet=nom_arrondissement_communes"

r = requests.get(link_json)
r

<Response [200]>

**[Réponses à des requêtes HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)**

In [ ]:
# JSON LOADS pour visualiser le JSON
data = json.loads(r.text)
data

In [ ]:
# PD JSON pour le transformer en dataframe
df = pd.json_normalize(r.json(), record_path='records')
df

### **2. Nettoyage des données**

In [ ]:
df.info()

In [ ]:
df.columns

Index(['datasetid', 'recordid', 'record_timestamp', 'fields.name',
       'fields.stationcode', 'fields.ebike', 'fields.mechanical',
       'fields.coordonnees_geo', 'fields.duedate', 'fields.numbikesavailable',
       'fields.numdocksavailable', 'fields.capacity', 'fields.is_renting',
       'fields.is_installed', 'fields.nom_arrondissement_communes',
       'fields.is_returning', 'geometry.type', 'geometry.coordinates'],
      dtype='object')

In [ ]:
# IS_RENTING ? IS_RETURNING ? IS_INSTALLED ?
df['fields.is_installed'].value_counts()

OUI    1441
NON      22
Name: fields.is_installed, dtype: int64

In [ ]:
df[df['fields.is_renting'] == 'NON']

In [ ]:
# liste des colonnes dont on aura besoin
columns_to_keep = ['record_timestamp', 'fields.stationcode', 'fields.name',
                   'fields.ebike', 'fields.mechanical',
                   'fields.numbikesavailable', 'fields.numdocksavailable',
                   'fields.is_renting', 'fields.nom_arrondissement_communes',
                   'fields.coordonnees_geo']

In [ ]:
df = df[columns_to_keep]
df

### **3. Cartographie**

In [ ]:
import folium

In [ ]:
df.columns

Index(['record_timestamp', 'fields.stationcode', 'fields.name', 'fields.ebike',
       'fields.mechanical', 'fields.numbikesavailable',
       'fields.numdocksavailable', 'fields.is_renting',
       'fields.nom_arrondissement_communes', 'fields.coordonnees_geo'],
      dtype='object')

In [ ]:
# On génère une carte centrée sur les coordonnées [48.860680473738334, 2.314848195584221]
m = folium.Map(location=[48.860680473738334, 2.314848195584221], zoom_start=15)

# On ajoute un marqueur par ligne du dataframe
for i in range(len(df)):
  coordinates = df.loc[i]['fields.coordonnees_geo']
  name = df.loc[i]['fields.name']
  available_ebikes = df.loc[i]['fields.ebike']
  available_mec_bikes = df.loc[i]['fields.mechanical']
  available_bikes_total = df.loc[i]['fields.numbikesavailable']
  available_docks = df.loc[i]['fields.numdocksavailable']
  is_renting = df.loc[i]['fields.is_renting']

  # Popup étoffé
  pop = folium.Popup(f"<b>{name}</b><br><br>{available_ebikes} vélos électriques disponibles<br>{available_mec_bikes} vélos mécaniques disponibles<br>{available_docks} places disponibles", max_width=250)
  # Marqueur bleu par défaut
  icon = folium.Icon(color='darkblue')

  # si station indisponible => marqueur gris
  if is_renting == 'NON':
    icon = folium.Icon(color='gray')
    pop = folium.Popup("<b>Station fermée</b>", max_width=250)

  # si moins de 3 vélos disponibles => marqueur orange
  if available_bikes_total <= 3:
    icon = folium.Icon(color='orange')


  folium.Marker(
      location=coordinates,
      popup=pop,
      icon=icon,
      ).add_to(m)

m

# **GeoCoding : requêtage avec l'[API Adresse](https://adresse.data.gouv.fr/api-doc/adresse) de data.gouv.fr**

### **1. Pour une adresse précise**

In [ ]:
# Exemple avec l'adresse 17 bis rue Delandine 69002
link = 'https://api-adresse.data.gouv.fr/search/?q=17+bis+Rue+Delandine&postcode=69002&limit=1'
r = requests.get(link)
r

<Response [200]>

In [ ]:
df_geo = pd.json_normalize(r.json(), record_path='features')
df_geo

,type,geometry.type,geometry.coordinates,properties.label,properties.score,properties.housenumber,properties.id,properties.name,properties.postcode,properties.citycode,properties.x,properties.y,properties.city,properties.district,properties.context,properties.type,properties.importance,properties.street
0,Feature,Point,"[4.826929, 45.746264]",17bis Rue Delandine 69002 Lyon,0.755825,17bis,69382_2120_00017_bis,17bis Rue Delandine,69002,69382,842030.68,6517939.49,Lyon,Lyon 2e Arrondissement,"69, Rhône, Auvergne-Rhône-Alpes",housenumber,0.66407,Rue Delandine


In [ ]:
# On observe que la requête se compose d'une partie fixe, suivie de l'adresse à chercher
# Une URL ne peut pas comporter de caractère espace " ",
# et il faut éviter si possible d'avoir des caractères spéciaux ou des accents

link_main = 'https://api-adresse.data.gouv.fr/search/?q='
adresse = '17 bis rue Delandine, 69002 Lyon'

# nettoyage léger :
adresse = adresse.replace(" ", "+")
adresse = adresse.replace(",", "")

link = link_main + adresse

print(link)

https://api-adresse.data.gouv.fr/search/?q=17+bis+rue+Delandine+69002+Lyon


### **2. Généralisation à toutes les adresses**

In [ ]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
from unidecode import unidecode

In [ ]:
# Nettoyage plus poussé :

# 1. Unidecode pour enlever les accents
adresse = '17 bis ? rue Délandine, 69002-Lyon'
adresse = unidecode(adresse)
adresse

'17 bis ? rue Delandine, 69002-Lyon'

In [ ]:
# 2. RegEx pour remplacer "tout ce qui n'est ni un mot ni un espace" par un caractère vide ' '
adresse = re.sub('[^\w\s]', ' ', adresse)
adresse

'17 bis   rue Delandine  69002 Lyon'

In [ ]:
# 3. RegEx pour remplacer tous les espaces par '+'
adresse = re.sub('[\s]+', '+', adresse)
adresse

'17+bis+rue+Delandine+69002+Lyon'

In [ ]:
# Crée ici une fonction qui :
# 1. transforme une adresse postale en URL de requête pour l'API Adresse,
# 2. effectue la requête et retourne les coordonnées :

# Étape 1
def API_adresse(adresse_postale):

  link_main = 'https://api-adresse.data.gouv.fr/search/?q='

  # Étape 1 : transformer l'adresse en URL
  adresse = unidecode(adresse_postale)
  adresse = re.sub('[^\w\s]', '', adresse)
  adresse = re.sub('[\s]+', '+', adresse)

  clean_link = link_main + adresse

  return clean_link

In [ ]:
# test de la fonction : ça marche !
adresse = '17 bis ? rue Délandine, 69002-Lyon'
clean_link = API_adresse(adresse)
clean_link

'https://api-adresse.data.gouv.fr/search/?q=17+bis+rue+Delandine+69002Lyon'

In [ ]:
r = requests.get(clean_link)
print(r)

df_geo = pd.json_normalize(r.json(), record_path='features')
df_geo

<Response [200]>


,type,geometry.type,geometry.coordinates,properties.label,properties.score,properties.housenumber,properties.id,properties.name,properties.postcode,properties.citycode,properties.x,properties.y,properties.city,properties.district,properties.context,properties.type,properties.importance,properties.street
0,Feature,Point,"[4.826929, 45.746264]",17bis Rue Delandine 69002 Lyon,0.725143,17bis,69382_2120_00017_bis,17bis Rue Delandine,69002,69382,842030.68,6517939.49,Lyon,Lyon 2e Arrondissement,"69, Rhône, Auvergne-Rhône-Alpes",housenumber,0.66407,Rue Delandine
1,Feature,Point,"[4.821543, 45.740182]",Rue Antoine Delandine 69002 Lyon,0.496312,NaN,69382_xswk3g,Rue Antoine Delandine,69002,69382,841627.61,6517254.58,Lyon,Lyon 2e Arrondissement,"69, Rhône, Auvergne-Rhône-Alpes",street,0.61328,Rue Antoine Delandine


In [ ]:
# Extraction des coordonnées
df_geo.loc[0]['geometry.coordinates'][::-1]

[45.746264, 4.826929]

In [ ]:
# Étape 2

def API_adresse(adresse_postale):

  link_main = 'https://api-adresse.data.gouv.fr/search/?q='

  # Étape 1 : transformer l'adresse en URL
  adresse = unidecode(adresse_postale)
  adresse = re.sub('[^\w\s]', '', adresse)
  adresse = re.sub('[\s]+', '+', adresse)

  clean_link = link_main + adresse

  # Étape 2 : effectuer la requête et retourner les coordonnées
  r = requests.get(clean_link)
  df_geo = pd.json_normalize(r.json(), record_path='features')

  coordinates = df_geo.loc[0]['geometry.coordinates'][::-1]

  return coordinates

In [ ]:
adresse = '17 bis ? rue Délandine, 69002-Lyon'
coordinates = API_adresse(adresse)
coordinates

In [ ]:
# Application au df des restaurants clermontois :

restaurants = pd.DataFrame([["Polypode","6 place du Champgil, Clermont-Ferrand, 63000"],
                            ["Jean-Claude Leclerc", "12 rue St-Adjutor, Clermont-Ferrand, 63000"],
                            ["L'Écureuil", "18 rue St-Adjutor, Clermont-Ferrand, 63000"],
                            ["Le Saint-Eutrope", "4 rue St-Eutrope, Clermont-Ferrand, 63000"]],
                           columns = ["nom", "adresse"])

restaurants

,nom,adresse
0,Polypode,"6 place du Champgil, Clermont-Ferrand, 63000"
1,Jean-Claude Leclerc,"12 rue St-Adjutor, Clermont-Ferrand, 63000"
2,L'Écureuil,"18 rue St-Adjutor, Clermont-Ferrand, 63000"
3,Le Saint-Eutrope,"4 rue St-Eutrope, Clermont-Ferrand, 63000"


In [ ]:
# création d'une nouvelle colonne coordonnées, générée grâce à notre fonction API_adresse
restaurants['coordonnees'] = restaurants['adresse'].apply(API_adresse)
restaurants

,nom,adresse,coordonnees
0,Polypode,"6 place du Champgil, Clermont-Ferrand, 63000","[45.778867, 3.079426]"
1,Jean-Claude Leclerc,"12 rue St-Adjutor, Clermont-Ferrand, 63000","[45.761745, 3.097876]"
2,L'Écureuil,"18 rue St-Adjutor, Clermont-Ferrand, 63000","[45.761389, 3.098109]"
3,Le Saint-Eutrope,"4 rue St-Eutrope, Clermont-Ferrand, 63000","[45.757716, 3.094736]"
